In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model, Input

def build_vae_encoder(input_dim, latent_dim=32, hidden_dims=[256, 128]):
    """
    Builds the encoder for a VAE.
    
    Args:
        input_dim (int): Flattened input size (e.g., 1500)
        latent_dim (int): Size of latent vector z
        hidden_dims (list): Sizes of hidden Dense layers
        
    Returns:
        encoder: Keras Model returning z_mean, z_log_var, and sampled z
    """

    inputs = Input(shape=(input_dim,), name="encoder_input")
    x = inputs

    # Hidden layers
    for i, dim in enumerate(hidden_dims):
        x = layers.Dense(dim, activation="relu", name=f"enc_dense_{i}")(x)

    # Output: mean and log variance
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

    # Sampling function using reparameterization trick
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = layers.Lambda(sampling, name="z")([z_mean, z_log_var])

    encoder = Model(inputs, [z_mean, z_log_var, z], name="vae_encoder")
    return encoder


In [ ]:
encoder = build_vae_encoder(input_dim=500, latent_dim=36, hidden_dims=[512, 256])
encoder.summary()


In [ ]:
from tensorflow.keras import layers, Model, Input

def build_vae_decoder(output_dim, latent_dim=32, hidden_dims=[128, 256]):
    """
    Builds the decoder part of a VAE.

    Args:
        output_dim (int): The number of output features (e.g. 1500 for ECG+PCG+ACC)
        latent_dim (int): The size of the latent vector z
        hidden_dims (list): List of Dense layer sizes in reverse order of encoder

    Returns:
        decoder: A Keras Model that maps z → reconstructed signal
    """

    latent_inputs = Input(shape=(latent_dim,), name="z_sampling")
    x = latent_inputs

    for i, dim in enumerate(hidden_dims):
        x = layers.Dense(dim, activation="leaky_relu", name=f"dec_dense_{i}")(x)

    # Final output layer
    outputs = layers.Dense(output_dim, activation="tanh", name="decoder_output")(x)

    decoder = Model(latent_inputs, outputs, name="vae_decoder")
    return decoder


In [ ]:
decode = build_vae_decoder(output_dim=500, latent_dim=36, hidden_dims=[256, 512])
decode.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model

class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def compile(self, optimizer, loss_fn, **kwargs):
        super(VAE, self).compile(**kwargs)
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.total_loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="recon_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, x):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            recon_loss = self.loss_fn(x, reconstruction)
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = recon_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "recon_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [ ]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

vae = VAE(encoder, decode)
vae.compile(optimizer=Adam(1e-3), loss_fn=MeanSquaredError())

#vae.fit(train_ds, epochs=30)



In [ ]:
import numpy as np 
data = np.load(r"C:\Users\abusu\Desktop\BME\ProJect64\ProJect64 System Architect\Data\ecgModel_data.npy")

In [ ]:
from tensorflow.keras.utils import plot_model

tt = plot_model(encoder, show_shapes=True, show_layer_names=True)
plot_model(decode, to_file="decoder.png", show_shapes=True, show_layer_names=True)


In [ ]:
from IPython.display import Image
Image(tt)


In [1]:
import numpy as np

In [2]:
model_data = np.load("/home/psyche/Documents/ProJect64/ProJect64 System Architect/Data/Model_data.npy")
print(model_data.shape)

(39879, 500)
